In [1]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

/usr/local/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [3]:
type(model.syn0)

numpy.ndarray

In [4]:
model.syn0.shape

(16490, 300)

In [5]:
model["flower"]

array([-0.02941535, -0.0513059 , -0.08721951,  0.00082673,  0.08641035,
        0.05378117,  0.12386004,  0.02094221, -0.00233172, -0.03997647,
       -0.05452802, -0.11389782, -0.05407133,  0.07904044, -0.03324372,
       -0.03749318, -0.00173147, -0.04767906, -0.08338775, -0.11164459,
       -0.15238291,  0.00488032,  0.10066137,  0.08054902,  0.02391852,
        0.05386177, -0.06786829, -0.04160012, -0.01197073, -0.04740219,
        0.00924124,  0.05392271, -0.05645082,  0.03756255, -0.09991515,
        0.02415238,  0.00039611, -0.02947001,  0.08141202, -0.00145684,
       -0.05961829,  0.03514636,  0.06249267, -0.09909658, -0.00758574,
       -0.02639271, -0.02670189, -0.02866178,  0.00241962,  0.13115166,
       -0.05429085,  0.10124787,  0.0662581 , -0.04968387, -0.03145637,
        0.0180555 ,  0.04574045, -0.06821576, -0.11828638, -0.0016918 ,
       -0.07409013, -0.01686165,  0.01236163, -0.05372545,  0.04862688,
        0.04574242, -0.05303368,  0.09391002,  0.08112244, -0.00

In [6]:
import numpy as np


In [23]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(model.index2word)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec



In [8]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0.
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1.
    return reviewFeatureVecs

In [24]:
import pandas as pd

import re

from bs4 import BeautifulSoup

import nltk.data
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)



tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3, encoding="utf8")
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3, encoding="utf8")
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3, encoding="utf8")

num_features = 300

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist( review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features)

print "creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [25]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )
forest = forest.fit( trainDataVecs, train["sentiment"] )
result = forest.predict(testDataVecs)


In [26]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result})
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3)

In [29]:
from sklearn.cluster import KMeans
import time

start = time.time()

word_vectors = model.syn0
num_clusters = word_vectors.shape[0]/5



In [30]:
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds"

Time taken for K Means clustering:  723.771988869 seconds


In [31]:
word_centroid_map = dict(zip(model.index2word, idx))

In [32]:
for cluster in xrange(0,10):
    print "\nCluster %d" % cluster
    
    words = []
    for i in xrange(0, len(word_centroid_map.values())):
        if(word_centroid_map.values()[i] == cluster):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'crane']

Cluster 1
[u'veneer', u'crux', u'ambiguity', u'interpretation', u'metaphysical', u'therein']

Cluster 2
[u'sterile', u'lumbering', u'schlocky', u'disposable', u'ghastly', u'hideous', u'soulless', u'disturbingly', u'grotesque', u'relentless']

Cluster 3
[u'memorial', u'bc', u'exile', u'berlin', u'rome', u'paris', u'vienna']

Cluster 4
[u'agreement', u'armored', u'ambush', u'airline', u'auto', u'expedition', u'automobile', u'interrogation']

Cluster 5
[u'ants', u'locusts', u'raptors', u'wasps', u'rats']

Cluster 6
[u'ahmed', u'andr']

Cluster 7
[u'titular', u'mai', u'eponymous', u'aptly']

Cluster 8
[u'rude', u'polite']

Cluster 9
[u'abandons', u'proposes', u'arranges', u'arrange']
